In [49]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
from termcolor import colored



In [50]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [51]:

# Import twitter data
print(colored("Loading train and test data", "green"))

tr = pd.read_csv("/content/drive/My Drive/big.csv")
te = pd.read_csv("/content/drive/My Drive/twt_#ifc.csv")

tr.full_text=tr.full_text.astype(str)
te.full_text=te.full_text.astype(str)
print(colored("... \nData loaded", "green"))


Loading train and test data
... 
Data loaded


In [52]:

# Tokenization
print(colored("Tokenizing and padding data", "green"))
tokenizer = Tokenizer(num_words = 2000, split = ' ')
tokenizer.fit_on_texts(tr['full_text'].astype(str).values)
train_tweets = tokenizer.texts_to_sequences(tr['full_text'].astype(str).values)
max_len = max([len(i) for i in train_tweets])
train_tweets = pad_sequences(train_tweets, maxlen = max_len)
test_tweets = tokenizer.texts_to_sequences(te['full_text'].astype(str).values)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)
print(colored("Tokenizing and padding complete", "yellow"))

Tokenizing and padding data
Tokenizing and padding complete


In [53]:
# Building the model
print(colored("Creating the GRU model", "yellow"))
model = Sequential()
model.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
model.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
model.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
model.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
model.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = False))

model.add(Dense(3, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Creating the GRU model
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_12 (Spatia (None, 37, 128)           0         
_________________________________________________________________
gru_6 (GRU)                  (None, 37, 256)           295680    
_________________________________________________________________
gru_7 (GRU)                  (None, 37, 256)           393984    
_________________________________________________________________
gru_8 (GRU)                  (None, 37, 256)           393984    
_________________________________________________________________
gru_9 (GRU)                  (None, 37, 256)           393984    
_________________________________________________________________
gru_10 (GRU)                 (

In [54]:
# Training the model
print(colored("Training the GRU model", "green"))
history = model.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the GRU model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 352s 17ms/step - loss: 3491986.1214 - accuracy: 0.4143
Epoch 2/3
20741/20741 [==============================] - 356s 17ms/step - loss: 360297.8788 - accuracy: 0.4250
Epoch 3/3
20741/20741 [==============================] - 353s 17ms/step - loss: 110699.0365 - accuracy: 0.4295


In [55]:
 print(colored("Testing the GRU model", "green"))

loss, accuracy = model.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))

score, accuracy = model.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))



Testing the GRU model
20741/20741 [==============================] - 80s 4ms/step
Train accuracy: 0.5352200865745544
1749/1749 [==============================] - 7s 4ms/step
Test accuracy: 0.5397369861602783
